In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/no_decoy_no_eve.csv')

In [4]:
import numpy as np

df = pd.read_csv("data/no_decoy_no_eve.csv")

# 1) Sifting
df_sift = df[df["alice_basis"] == df["bob_basis"]].copy()
N_total = len(df)
N_match = len(df_sift)

# 2) QBER on sifted data
df_sift["error"] = (df_sift["alice_value"] != df_sift["bob_bit"]).astype(int)
N_err = int(df_sift["error"].sum())
QBER = N_err / N_match if N_match else np.nan

# Optional: binomial standard error
QBER_sigma = np.sqrt(QBER*(1-QBER)/N_match) if N_match else np.nan

N_total, N_match, N_err, QBER, QBER_sigma


(52, 31, 0, 0.0, 0.0)

In [5]:
# 3) Construct the sifted keys (as strings for readability)
K_A = "".join(df_sift["alice_value"].astype(int).astype(str).tolist())
K_B = "".join(df_sift["bob_bit"].astype(int).astype(str).tolist())

K_A, K_B, (K_A == K_B)


('1010100111000111000011010110111', '1010100111000111000011010110111', True)

In [8]:
def bb84_sift_and_qber(csv_path: str):
    """
    Load one BB84 run (Alice/Bob; optional Eve column) and compute:
    - N_total, N_match (sifted length), N_err, QBER, QBER_sigma
    - sifted keys K_A, K_B and whether they are identical
    """
    df = pd.read_csv(csv_path)

    # sanity checks (optional but helpful)
    assert set(df["alice_basis"]).issubset({"+", "x"})
    assert set(df["bob_basis"]).issubset({"+", "x"})
    assert set(df["alice_value"]).issubset({0, 1})
    assert set(df["bob_bit"]).issubset({0, 1})

    # 1) sifting
    df_sift = df[df["alice_basis"] == df["bob_basis"]].copy()

    N_total = len(df)
    N_match = len(df_sift)

    # 2) errors + QBER (on sifted bits only)
    df_sift["error"] = (df_sift["alice_value"] != df_sift["bob_bit"]).astype(int)
    N_err = int(df_sift["error"].sum())

    QBER = (N_err / N_match) if N_match else np.nan
    QBER_sigma = (np.sqrt(QBER * (1 - QBER) / N_match)) if N_match else np.nan

    # 3) sifted keys
    K_A = "".join(df_sift["alice_value"].astype(int).astype(str).tolist())
    K_B = "".join(df_sift["bob_bit"].astype(int).astype(str).tolist())
    keys_equal = (K_A == K_B)

    out = {
        "N_total": N_total,
        "N_match": N_match,
        "N_err": N_err,
        "QBER": QBER,
        "QBER_sigma": QBER_sigma,
        "K_A": K_A,
        "K_B": K_B,
        "keys_equal": keys_equal,
        "df_sift": df_sift,  # keep for later (e.g. plots or inspection)
    }
    return out

# ---- Run it on your BB84 with Eve file ----
res_eve = bb84_sift_and_qber("data/no_decoy_w_eve.csv")

print("BB84 with Eve")
print(f"N_total   = {res_eve['N_total']}")
print(f"N_match   = {res_eve['N_match']}")
print(f"N_err     = {res_eve['N_err']}")
print(f"QBER      = {res_eve['QBER']:.4f}")
print(f"sigma(Q)  = {res_eve['QBER_sigma']:.4f}")
print(f"K_A == K_B? {res_eve['keys_equal']}")

# Optional: look at the first few sifted events and errors
res_eve["df_sift"][["i", "alice_basis", "alice_value", "bob_bit", "error"]].head(20)


BB84 with Eve
N_total   = 52
N_match   = 26
N_err     = 2
QBER      = 0.0769
sigma(Q)  = 0.0523
K_A == K_B? False


,i,alice_basis,alice_value,bob_bit,error
2,3,+,0,0,0
4,5,+,1,1,0
6,7,x,1,1,0
8,9,x,0,0,0
9,10,x,1,1,0
11,12,x,0,0,0
12,13,+,0,0,0
13,14,x,1,1,0
14,15,x,0,0,0
17,18,x,1,1,0


In [9]:
from math import comb

n=26; p=0.25
prob = sum(comb(n,k)*(p**k)*((1-p)**(n-k)) for k in range(0,3))
prob


0.025837325396998345

In [11]:
df = pd.read_csv("data/no_decoy_w_eve.csv")
df_sift = df[df["alice_basis"] == df["bob_basis"]].copy()

# how often did Eve use the same basis as Alice (on sifted events)
p_eve_match = (df_sift["eve_basis"] == df_sift["alice_basis"]).mean()
N = len(df_sift)
print("Eve basis match fraction (on sifted events):", p_eve_match, " N=", N)


Eve basis match fraction (on sifted events): 0.6538461538461539  N= 26


In [12]:
df_sift["eve_matches_alice"] = (df_sift["eve_basis"] == df_sift["alice_basis"])
df_sift["error"] = (df_sift["alice_value"] != df_sift["bob_bit"])

print("QBER when Eve basis matches Alice:", df_sift[df_sift["eve_matches_alice"]]["error"].mean())
print("QBER when Eve basis mismatches Alice:", df_sift[~df_sift["eve_matches_alice"]]["error"].mean())
print("Counts:", df_sift["eve_matches_alice"].value_counts())


QBER when Eve basis matches Alice: 0.0
QBER when Eve basis mismatches Alice: 0.2222222222222222
Counts: eve_matches_alice
True     17
False     9
Name: count, dtype: int64


In [13]:
def summarize_bb84(run_name, res):
    return {
        "run": run_name,
        "N_total": res["N_total"],
        "N_match": res["N_match"],
        "N_err": res["N_err"],
        "QBER": res["QBER"],
        "QBER_sigma": res["QBER_sigma"],
        "keys_equal": res["keys_equal"],
    }

res_noeve = bb84_sift_and_qber("data/no_decoy_no_eve.csv")
res_eve   = bb84_sift_and_qber("data/no_decoy_w_eve.csv")

summary = pd.DataFrame([
    summarize_bb84("BB84 no Eve", res_noeve),
    summarize_bb84("BB84 with Eve", res_eve),
])

summary


,run,N_total,N_match,N_err,QBER,QBER_sigma,keys_equal
0,BB84 no Eve,52,31,0,0.000000,0.000000,True
1,BB84 with Eve,52,26,2,0.076923,0.052259,False


In [14]:
import pandas as pd
import numpy as np

def decoy_sift_and_qber(csv_path: str):
    """
    Load one Decoy run (Alice trits + Bob bits; optional Eve basis) and compute:
    - N_total, N_match (sifted length)
    - QBER using intended bit mapping from trit: 1->1, {0,2}->0
    - Optional: diagnostic splits (by trit, by Eve basis match)
    """
    df = pd.read_csv(csv_path)

    # sanity checks
    assert set(df["alice_basis"]).issubset({"+", "x"})
    assert set(df["bob_basis"]).issubset({"+", "x"})
    assert set(df["alice_value"]).issubset({0, 1, 2})
    assert set(df["bob_bit"]).issubset({0, 1})

    # 1) sifting
    df_sift = df[df["alice_basis"] == df["bob_basis"]].copy()
    N_total = len(df)
    N_match = len(df_sift)

    # 2) map Alice trit -> intended bit (per manual signal-state mapping)
    # trit 1 corresponds to polarization encoding bit=1 in both bases; trit 0 and 2 -> bit=0
    df_sift["alice_intended_bit"] = (df_sift["alice_value"] == 1).astype(int)

    # 3) QBER on sifted data
    df_sift["error"] = (df_sift["alice_intended_bit"] != df_sift["bob_bit"]).astype(int)
    N_err = int(df_sift["error"].sum())
    QBER = (N_err / N_match) if N_match else np.nan
    QBER_sigma = (np.sqrt(QBER * (1 - QBER) / N_match)) if N_match else np.nan

    out = {
        "N_total": N_total,
        "N_match": N_match,
        "N_err": N_err,
        "QBER": QBER,
        "QBER_sigma": QBER_sigma,
        "df_sift": df_sift
    }

    # 4) optional diagnostics if Eve basis exists
    if "eve_basis" in df.columns:
        df_sift["eve_matches_alice"] = (df_sift["eve_basis"] == df_sift["alice_basis"])
        out["eve_basis_match_fraction_on_sifted"] = df_sift["eve_matches_alice"].mean()
        out["QBER_eve_match"] = df_sift[df_sift["eve_matches_alice"]]["error"].mean() if (df_sift["eve_matches_alice"].any()) else np.nan
        out["QBER_eve_mismatch"] = df_sift[~df_sift["eve_matches_alice"]]["error"].mean() if ((~df_sift["eve_matches_alice"]).any()) else np.nan
        out["eve_match_counts"] = df_sift["eve_matches_alice"].value_counts().to_dict()

    # 5) optional diagnostics by trit
    out["QBER_by_trit"] = df_sift.groupby("alice_value")["error"].mean().to_dict()
    out["counts_by_trit"] = df_sift["alice_value"].value_counts().sort_index().to_dict()

    return out


# --- Run it on both Decoy datasets ---
res_decoy_noeve = decoy_sift_and_qber("data/decoy_no_eve.csv")
res_decoy_eve   = decoy_sift_and_qber("data/decoy_w_eve.csv")

print("Decoy no Eve")
print("N_match =", res_decoy_noeve["N_match"], "N_err =", res_decoy_noeve["N_err"],
      "QBER =", f"{res_decoy_noeve['QBER']:.4f}", "sigma =", f"{res_decoy_noeve['QBER_sigma']:.4f}")
print("Counts by trit:", res_decoy_noeve["counts_by_trit"])
print("QBER by trit:", res_decoy_noeve["QBER_by_trit"])

print("\nDecoy with Eve")
print("N_match =", res_decoy_eve["N_match"], "N_err =", res_decoy_eve["N_err"],
      "QBER =", f"{res_decoy_eve['QBER']:.4f}", "sigma =", f"{res_decoy_eve['QBER_sigma']:.4f}")
if "eve_basis_match_fraction_on_sifted" in res_decoy_eve:
    print("Eve basis match fraction (sifted):", res_decoy_eve["eve_basis_match_fraction_on_sifted"])
    print("Eve match counts:", res_decoy_eve["eve_match_counts"])
    print("QBER when Eve matches Alice:", res_decoy_eve["QBER_eve_match"])
    print("QBER when Eve mismatches Alice:", res_decoy_eve["QBER_eve_mismatch"])
print("Counts by trit:", res_decoy_eve["counts_by_trit"])
print("QBER by trit:", res_decoy_eve["QBER_by_trit"])


Decoy no Eve
N_match = 21 N_err = 0 QBER = 0.0000 sigma = 0.0000
Counts by trit: {0: 6, 1: 10, 2: 5}
QBER by trit: {0: 0.0, 1: 0.0, 2: 0.0}

Decoy with Eve
N_match = 29 N_err = 6 QBER = 0.2069 sigma = 0.0752
Eve basis match fraction (sifted): 0.5517241379310345
Eve match counts: {True: 16, False: 13}
QBER when Eve matches Alice: 0.0
QBER when Eve mismatches Alice: 0.46153846153846156
Counts by trit: {0: 13, 1: 11, 2: 5}
QBER by trit: {0: 0.23076923076923078, 1: 0.2727272727272727, 2: 0.0}
